In [1]:
import tensorflow as tf

# Load the model
cnn = tf.keras.models.load_model('my_model')


In [2]:
import numpy as np
from keras.preprocessing import image
for i in range(1,5):
    test_image = image.load_img(f"dataset/single_prediction/cat_or_dog_{i}.jpg", target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    print(result)
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    print(prediction)

1/1 [==============================] - 8s 8s/step
[[1.]]
dog
1/1 [==============================] - 0s 39ms/step
[[1.]]
dog
1/1 [==============================] - 0s 43ms/step
[[0.]]
cat
1/1 [==============================] - 0s 33ms/step
[[1.]]
dog


In [3]:
# Load the model
cnn = tf.keras.models.load_model('cat_or_dog_model.h5')

In [4]:
import numpy as np
from keras.preprocessing import image
for i in range(1,5):
    test_image = image.load_img(f"dataset/single_prediction/cat_or_dog_{i}.jpg", target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    print(result)
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    print(prediction)

1/1 [==============================] - 0s 384ms/step
[[1.]]
dog
1/1 [==============================] - 0s 33ms/step
[[1.]]
dog
1/1 [==============================] - 0s 45ms/step
[[0.]]
cat
1/1 [==============================] - 0s 51ms/step
[[1.]]
dog


In [5]:
import cv2
import numpy as np
import tensorflow as tf

# Assuming your trained model is stored in the variable `cnn`
model = cnn

# Define class indices (this should match how you defined them during training)
class_indices = {'cat': 0, 'dog': 1}
index_to_class = {v: k for k, v in class_indices.items()}

def preprocess_frame(frame, img_size=(64, 64)):
    img_array = cv2.resize(frame, img_size)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

def predict_frame(model, frame):
    preprocessed_frame = preprocess_frame(frame)
    prediction = model.predict(preprocessed_frame)
    predicted_class_index = int(np.round(prediction[0][0]))  # Assuming binary classification with sigmoid activation
    predicted_class_label = index_to_class[predicted_class_index]
    return predicted_class_label

# Load the pre-trained MobileNet SSD model for object detection
prototxt_path = 'deploy.prototxt'
caffemodel_path = 'mobilenet_iter_73000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Start video capture from camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # Confidence threshold
            # Extract the index of the class label from the detections
            idx = int(detections[0, 0, i, 1])

            # If the detected object is a dog or cat
            if idx in [15, 3]:  # Adjust this based on the object detector's class labels for 'dog' and 'cat'
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Crop the detected region
                cropped_frame = frame[startY:endY, startX:endX]

                # Classify the cropped region using the CNN model
                predicted_class_label = predict_frame(model, cropped_frame)

                # Draw bounding box and label on the frame
                label = f"{predicted_class_label}: {confidence:.2f}"
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow('Live Dog/Cat Classifier', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 16ms/step


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
